In [ ]:
'''
The steps of project:

Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
Apply a distortion correction to raw images.
Use color transforms, gradients, etc., to create a thresholded binary image.
Apply a perspective transform to rectify binary image ("birds-eye view").
Detect lane pixels and fit to find the lane boundary.
Determine the curvature of the lane and vehicle position with respect to center.
Warp the detected lane boundaries back onto the original image.
Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position
'''
# Find corners and undistort
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
import pickle

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        print ('working on ', fname)
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners, ret)
        write_name = 'examples/corners_found' + str(idx) + '.jpg'
        cv2.imwrite(write_name, img)

# reference image
img = cv2.imread('camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

# do camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
print ('undistort ', fname)
undistorted = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('output_images/calibration1_undistorted.jpg', undistorted)

#save the camera calibrate result
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/calibration_pickle.p", "wb" ))

In [ ]:
# Find threshold image using sobel and color threshold
import numpy as np
import cv2
import pickle

# Define Helper functions

# Function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
def abs_sobel_thresh(img, orient='x', thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

# Color threshold
def color_threshold(image, sthresh=(0, 255), vthresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    s_binary = np.zeros_like(S)
    s_binary[(S >= sthresh[0]) & (S <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    V = hsv[:,:,2]
    v_binary = np.zeros_like(V)
    v_binary[(V >= vthresh[0]) & (V <= vthresh[1])] = 1
    
    output = np.zeros_like(S)
    output[(s_binary == 1) & (v_binary == 1)] = 1
    return output

# Apply Sobel x and y, then compute the magnitude of the gradient
# and applies a threshold
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelxy = np.sqrt(sobelx**2 + sobely**2)
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return sbinary

# Apply Sobel x and y, then compute the direction of the gradient
# and apply a threshold.
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    dir = np.arctan2(abs_sobely, abs_sobelx)
    sbinary = np.zeros_like(dir)
    sbinary[(dir >= thresh[0]) & (dir <= thresh[1])] = 1
    return sbinary

#Read saved imgpoints and objpoints
dist_pickle = pickle.load( open("camera_cal/calibration_pickle.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# reference image
img = cv2.imread('test_images/test3.jpg')

img = cv2.undistort(img, mtx, dist, None, mtx)

cv2.imwrite('output_images/test3_undistorted.jpg', img)
    
#process image, find sobel and color threshold and draw lines
processimage = np.zeros_like(img[:,:,0])
gradx = abs_sobel_thresh(img, orient='x', thresh=(10,255))
grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
processimage[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255

cv2.imwrite('output_images/test3_find_lines_by_threshold.jpg', processimage)

In [9]:
# Do prespective transform

import numpy as np
import cv2
import pickle

# reference image
img = cv2.imread('output_images/test3_find_lines_by_threshold.jpg')

#perspective transform
img_size = (img.shape[1], img.shape[0])

# Use hard coded values for proper understanding as first step
bottom_left = [220,720]
bottom_right = [1110, 720]
top_left = [570, 470]
top_right = [722, 470]
src = np.float32([bottom_left,bottom_right,top_right,top_left])

# Cover main area
bottom_left = [320,720]
bottom_right = [920, 720]
top_left = [320, 0]
top_right = [920, 0]
dst = np.float32([bottom_left,bottom_right,top_right,top_left])

M = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

cv2.imwrite('output_images/test3_prespective_transform.jpg', warped)

True

In [7]:
# Find lane line pixels and window centeriods
import numpy as np
import cv2
import pickle

def abs_sobel_thresh(img, orient='x', thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

# Color threshold
def color_threshold(image, sthresh=(0, 255), vthresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    s_binary = np.zeros_like(S)
    s_binary[(S >= sthresh[0]) & (S <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    V = hsv[:,:,2]
    v_binary = np.zeros_like(V)
    v_binary[(V >= vthresh[0]) & (V <= vthresh[1])] = 1
    
    output = np.zeros_like(S)
    output[(s_binary == 1) & (v_binary == 1)] = 1
    return output

def track_line(warped, window_width, window_height):
    margin = 25
    ym_per_pix = 10/720 # meters per pixel in y dimension
    xm_per_pix = 4/384 # meters per pixel in x dimension
    smooth_factor = 12
    
    recent_centers = []      
    window_centroids = [] #Store the window centroid positions
    window = np.ones(window_width) # Window template that will be used for convolution

    # Sum quater bottom of image to get slice
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window, l_sum)) - window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window, r_sum)) - window_width/2 + int(warped.shape[1]/2)

    # Add first layer
    window_centroids.append((l_center, r_center))

    # Go through each layer looking for maximum pixel locations
    for level in range(1, (int)(warped.shape[0]/window_height)):
        # Convolve the window into vertical slice
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find best left centroid using past left center
        # Use window_width/2 as offset as signal reference is on right side
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin, warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset

        # Find best right centroid by using past right center
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin, warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset

        # Add layer
        window_centroids.append((l_center, r_center))

    recent_centers.append(window_centroids)

    # Return average values of the line centers
    return np.average(recent_centers[-smooth_factor:], axis=0)

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]- (level+1)*height):int(img_ref.shape[0]-level*height), max(0, int(center-width)):min(int(center+width), img_ref.shape[1])] = 1
    return output

#Read saved imgpoints and objpoints
dist_pickle = pickle.load( open("camera_cal/calibration_pickle.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# reference image
img = cv2.imread('test_images/test3.jpg')

img = cv2.undistort(img, mtx, dist, None, mtx)
    
#process image, find sobel and color threshold and draw lines
processimage = np.zeros_like(img[:,:,0])
gradx = abs_sobel_thresh(img, orient='x', thresh=(10,255))
grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
processimage[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255

#perspective transform
img_size = (img.shape[1], img.shape[0])

# Use hard coded values for proper understanding as first step
bottom_left = [220,720]
bottom_right = [1110, 720]
top_left = [570, 470]
top_right = [722, 470]
src = np.float32([bottom_left,bottom_right,top_right,top_left])

# Cover main area
bottom_left = [320,720]
bottom_right = [920, 720]
top_left = [320, 0]
top_right = [920, 0]
dst = np.float32([bottom_left,bottom_right,top_right,top_left])

M = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(processimage, M, img_size, flags=cv2.INTER_LINEAR)

window_width = 25
window_height = 80
window_centroids = track_line(warped, window_width, window_height)
    
l_points = np.zeros_like(warped)
r_points = np.zeros_like(warped)

# Points used to find right and left lanes
rightx = []
leftx = []

# Go through each level and draw windows
for level in range(0, len(window_centroids)):
    # Add center points to list of left, right lane points
    leftx.append(window_centroids[level][0])
    rightx.append(window_centroids[level][1])
    # Window mask is drawing window areas
    l_mask = window_mask(window_width, window_height, warped, window_centroids[level][0], level)
    r_mask = window_mask(window_width, window_height, warped, window_centroids[level][1], level)
    # Add graphic points
    l_points[(l_points == 255) | ((l_mask == 1))] = 255
    r_points[(r_points == 255) | ((r_mask == 1))] = 255

# Draw the results
template = np.array(r_points+l_points,np.uint8)
zero_channel = np.zeros_like(template)
template = np.array(cv2.merge((zero_channel, template, zero_channel)), np.uint8) # Green pixels
warpage = np.array(cv2.merge((warped, warped, warped)), np.uint8) # Original road pixel in three color channels
result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # Overlay the original image with road results

cv2.imwrite('output_images/test3_window_centriod.jpg', result)

True

In [2]:
# Find Lane Boundries
import numpy as np
import cv2
import pickle

def abs_sobel_thresh(img, orient='x', thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

# Color threshold
def color_threshold(image, sthresh=(0, 255), vthresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    s_binary = np.zeros_like(S)
    s_binary[(S >= sthresh[0]) & (S <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    V = hsv[:,:,2]
    v_binary = np.zeros_like(V)
    v_binary[(V >= vthresh[0]) & (V <= vthresh[1])] = 1
    
    output = np.zeros_like(S)
    output[(s_binary == 1) & (v_binary == 1)] = 1
    return output

def track_line(warped, window_width, window_height):
    margin = 25
    ym_per_pix = 10/720 # meters per pixel in y dimension
    xm_per_pix = 4/384 # meters per pixel in x dimension
    smooth_factor = 12
    
    recent_centers = []      
    window_centroids = [] #Store the window centroid positions
    window = np.ones(window_width) # Window template that will be used for convolution

    # Sum quater bottom of image to get slice
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window, l_sum)) - window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window, r_sum)) - window_width/2 + int(warped.shape[1]/2)

    # Add first layer
    window_centroids.append((l_center, r_center))

    # Go through each layer looking for maximum pixel locations
    for level in range(1, (int)(warped.shape[0]/window_height)):
        # Convolve the window into vertical slice
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find best left centroid using past left center
        # Use window_width/2 as offset as signal reference is on right side
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin, warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset

        # Find best right centroid by using past right center
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin, warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset

        # Add layer
        window_centroids.append((l_center, r_center))

    recent_centers.append(window_centroids)

    # Return average values of the line centers
    return np.average(recent_centers[-smooth_factor:], axis=0)

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]- (level+1)*height):int(img_ref.shape[0]-level*height), max(0, int(center-width)):min(int(center+width), img_ref.shape[1])] = 1
    return output

#Read saved imgpoints and objpoints
dist_pickle = pickle.load( open("camera_cal/calibration_pickle.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# reference image
img = cv2.imread('test_images/test3.jpg')

img = cv2.undistort(img, mtx, dist, None, mtx)
    
#process image, find sobel and color threshold and draw lines
processimage = np.zeros_like(img[:,:,0])
gradx = abs_sobel_thresh(img, orient='x', thresh=(10,255))
grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
processimage[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255

#perspective transform
img_size = (img.shape[1], img.shape[0])

# Use hard coded values for proper understanding as first step
bottom_left = [220,720]
bottom_right = [1110, 720]
top_left = [570, 470]
top_right = [722, 470]
src = np.float32([bottom_left,bottom_right,top_right,top_left])

# Cover main area
bottom_left = [320,720]
bottom_right = [920, 720]
top_left = [320, 0]
top_right = [920, 0]
dst = np.float32([bottom_left,bottom_right,top_right,top_left])

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
warped = cv2.warpPerspective(processimage, M, img_size, flags=cv2.INTER_LINEAR)

window_width = 25
window_height = 80
window_centroids = track_line(warped, window_width, window_height)
    
l_points = np.zeros_like(warped)
r_points = np.zeros_like(warped)

# Points used to find right and left lanes
rightx = []
leftx = []

# Go through each level and draw windows
for level in range(0, len(window_centroids)):
    # Add center points to list of left, right lane points
    leftx.append(window_centroids[level][0])
    rightx.append(window_centroids[level][1])
    # Window mask is drawing window areas
    l_mask = window_mask(window_width, window_height, warped, window_centroids[level][0], level)
    r_mask = window_mask(window_width, window_height, warped, window_centroids[level][1], level)
    # Add graphic points
    l_points[(l_points == 255) | ((l_mask == 1))] = 255
    r_points[(r_points == 255) | ((r_mask == 1))] = 255

# Draw the results
template = np.array(r_points+l_points,np.uint8)
zero_channel = np.zeros_like(template)
template = np.array(cv2.merge((zero_channel, template, zero_channel)), np.uint8) # Green pixels
warpage = np.array(cv2.merge((warped, warped, warped)), np.uint8) # Original road pixel in three color channels
result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # Overlay the original image with road results

# Fit the lane boundries to the left and right center positions found
yvals = range(0, warped.shape[0])

res_yvals = np.arange(warped.shape[0]-(window_height/2),0,-window_height)

left_fit = np.polyfit(res_yvals, leftx, 2)
left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
left_fitx = np.array(left_fitx, np.int32)

right_fit = np.polyfit(res_yvals, rightx, 2)
right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
right_fitx = np.array(right_fitx, np.int32)

left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,left_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
inner_lane = np.array(list(zip(np.concatenate((left_fitx+window_width/2,right_fitx[::-1]-window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)

road = np.zeros_like(img)
road_bkg = np.zeros_like(img)
cv2.fillPoly(road, [left_lane], color=[255, 0, 0])
cv2.fillPoly(road, [right_lane], color=[0,0,255])
cv2.fillPoly(road_bkg, [left_lane], color=[255, 255, 255])
cv2.fillPoly(road_bkg, [right_lane], color=[255, 255, 255])
cv2.fillPoly(road, [inner_lane], color=[0,255,0])

road_warped = cv2.warpPerspective(road, Minv, img_size, flags=cv2.INTER_LINEAR)
road_warped_bkg = cv2.warpPerspective(road_bkg, Minv, img_size, flags=cv2.INTER_LINEAR)

base = cv2.addWeighted(img, 1.0, road_warped_bkg, -1.0, 0.0)
result = cv2.addWeighted(base, 1.0, road_warped, 1.0, 0.0)
    
cv2.imwrite('output_images/test3_lane_boundries.jpg', result)

True

In [3]:
# Calculate Camera Center and Curve
import numpy as np
import cv2
import pickle

def abs_sobel_thresh(img, orient='x', thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

# Color threshold
def color_threshold(image, sthresh=(0, 255), vthresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    s_binary = np.zeros_like(S)
    s_binary[(S >= sthresh[0]) & (S <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    V = hsv[:,:,2]
    v_binary = np.zeros_like(V)
    v_binary[(V >= vthresh[0]) & (V <= vthresh[1])] = 1
    
    output = np.zeros_like(S)
    output[(s_binary == 1) & (v_binary == 1)] = 1
    return output

def track_line(warped, window_width, window_height):
    margin = 25
    ym_per_pix = 10/720 # meters per pixel in y dimension
    xm_per_pix = 4/384 # meters per pixel in x dimension
    smooth_factor = 12
    
    recent_centers = []      
    window_centroids = [] #Store the window centroid positions
    window = np.ones(window_width) # Window template that will be used for convolution

    # Sum quater bottom of image to get slice
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window, l_sum)) - window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window, r_sum)) - window_width/2 + int(warped.shape[1]/2)

    # Add first layer
    window_centroids.append((l_center, r_center))

    # Go through each layer looking for maximum pixel locations
    for level in range(1, (int)(warped.shape[0]/window_height)):
        # Convolve the window into vertical slice
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find best left centroid using past left center
        # Use window_width/2 as offset as signal reference is on right side
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin, warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset

        # Find best right centroid by using past right center
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin, warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset

        # Add layer
        window_centroids.append((l_center, r_center))

    recent_centers.append(window_centroids)

    # Return average values of the line centers
    return np.average(recent_centers[-smooth_factor:], axis=0)

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]- (level+1)*height):int(img_ref.shape[0]-level*height), max(0, int(center-width)):min(int(center+width), img_ref.shape[1])] = 1
    return output

#Read saved imgpoints and objpoints
dist_pickle = pickle.load( open("camera_cal/calibration_pickle.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# reference image
img = cv2.imread('test_images/test3.jpg')

img = cv2.undistort(img, mtx, dist, None, mtx)
    
#process image, find sobel and color threshold and draw lines
processimage = np.zeros_like(img[:,:,0])
gradx = abs_sobel_thresh(img, orient='x', thresh=(10,255))
grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
c_binary = color_threshold(img, sthresh=(100, 255), vthresh=(50,255))
processimage[((gradx == 1) & (grady == 1) | (c_binary == 1))] = 255

#perspective transform
img_size = (img.shape[1], img.shape[0])

# Use hard coded values for proper understanding as first step
bottom_left = [220,720]
bottom_right = [1110, 720]
top_left = [570, 470]
top_right = [722, 470]
src = np.float32([bottom_left,bottom_right,top_right,top_left])

# Cover main area
bottom_left = [320,720]
bottom_right = [920, 720]
top_left = [320, 0]
top_right = [920, 0]
dst = np.float32([bottom_left,bottom_right,top_right,top_left])

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
warped = cv2.warpPerspective(processimage, M, img_size, flags=cv2.INTER_LINEAR)

window_width = 25
window_height = 80
window_centroids = track_line(warped, window_width, window_height)
    
l_points = np.zeros_like(warped)
r_points = np.zeros_like(warped)

# Points used to find right and left lanes
rightx = []
leftx = []

# Go through each level and draw windows
for level in range(0, len(window_centroids)):
    # Add center points to list of left, right lane points
    leftx.append(window_centroids[level][0])
    rightx.append(window_centroids[level][1])
    # Window mask is drawing window areas
    l_mask = window_mask(window_width, window_height, warped, window_centroids[level][0], level)
    r_mask = window_mask(window_width, window_height, warped, window_centroids[level][1], level)
    # Add graphic points
    l_points[(l_points == 255) | ((l_mask == 1))] = 255
    r_points[(r_points == 255) | ((r_mask == 1))] = 255

# Draw the results
template = np.array(r_points+l_points,np.uint8)
zero_channel = np.zeros_like(template)
template = np.array(cv2.merge((zero_channel, template, zero_channel)), np.uint8) # Green pixels
warpage = np.array(cv2.merge((warped, warped, warped)), np.uint8) # Original road pixel in three color channels
result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # Overlay the original image with road results

# Fit the lane boundries to the left and right center positions found
yvals = range(0, warped.shape[0])

res_yvals = np.arange(warped.shape[0]-(window_height/2),0,-window_height)

left_fit = np.polyfit(res_yvals, leftx, 2)
left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
left_fitx = np.array(left_fitx, np.int32)

right_fit = np.polyfit(res_yvals, rightx, 2)
right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
right_fitx = np.array(right_fitx, np.int32)

left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,left_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
inner_lane = np.array(list(zip(np.concatenate((left_fitx+window_width/2,right_fitx[::-1]-window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)

road = np.zeros_like(img)
road_bkg = np.zeros_like(img)
cv2.fillPoly(road, [left_lane], color=[255, 0, 0])
cv2.fillPoly(road, [right_lane], color=[0,0,255])
cv2.fillPoly(road_bkg, [left_lane], color=[255, 255, 255])
cv2.fillPoly(road_bkg, [right_lane], color=[255, 255, 255])
cv2.fillPoly(road, [inner_lane], color=[0,255,0])

road_warped = cv2.warpPerspective(road, Minv, img_size, flags=cv2.INTER_LINEAR)
road_warped_bkg = cv2.warpPerspective(road_bkg, Minv, img_size, flags=cv2.INTER_LINEAR)

base = cv2.addWeighted(img, 1.0, road_warped_bkg, -1.0, 0.0)
result = cv2.addWeighted(base, 1.0, road_warped, 1.0, 0.0)

ym_per_pix = 10/720 # meters per pixel in y dimension
xm_per_pix = 4/384 # meters per pixel in x dimension

curve_fit_cr = np.polyfit(np.array(res_yvals, np.float32)*ym_per_pix, np.array(leftx, np.float32)*xm_per_pix, 2)
curved = ((1 + (2*curve_fit_cr[0]*yvals[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5) /np.absolute(2*curve_fit_cr[0])

# Calculate the offset of the car on the road
camera_center = (left_fitx[-1] + right_fitx[-1])/2
center_diff = (camera_center-warped.shape[1]/2)*xm_per_pix
side_pos = 'left'
if center_diff <= 0:
    side_pos = 'right'

# Draw the text showing curvature, offset and speed
cv2.putText(result, 'Radius of the Curvature = ' + str(round(curved, 3))+'(m)',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff, 3)))+'m '+side_pos+' of center', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
cv2.imwrite('output_images/test3_camera_center_and_curve.jpg', result)

True

In [14]:
# Process Project Video
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2
import pickle
import glob
import matplotlib.pyplot as plt

#Read saved imgpoints and objpoints
dist_pickle = pickle.load( open("camera_cal/calibration_pickle.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.
def abs_sobel_thresh(img, orient='x', thresh=(30, 100)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary
    
def color_threshold(image, sthresh=(170, 255), vthresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    s_binary = np.zeros_like(S)
    s_binary[(S >= sthresh[0]) & (S <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    V = hsv[:,:,2]
    v_binary = np.zeros_like(V)
    v_binary[(V >= vthresh[0]) & (V <= vthresh[1])] = 1
    
    output = np.zeros_like(S)
    #output[(s_binary == 1) & (v_binary == 1)] = 1
    output[(s_binary == 1)] = 1
    return s_binary

# Apply Sobel x and y, then compute the magnitude of the gradient
# and applies a threshold
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelxy = np.sqrt(sobelx**2 + sobely**2)
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return sbinary

# Apply Sobel x and y, then compute the direction of the gradient
# and apply a threshold.
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    dir = np.arctan2(abs_sobely, abs_sobelx)
    sbinary = np.zeros_like(dir)
    sbinary[(dir >= thresh[0]) & (dir <= thresh[1])] = 1
    return sbinary

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]- (level+1)*height):int(img_ref.shape[0]-level*height), max(0, int(center-width)):min(int(center+width), img_ref.shape[1])] = 1
    return output

def hist(img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set an `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    return histogram

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = hist(binary_warped)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 13
    # Set the width of the windows +/- margin
    margin = 90
    # Set minimum number of pixels found to recenter window
    minpix = 40

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 3) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 3) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def fit_poly(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return left_fit, right_fit

def search_around_poly(binary_warped):
    
    # HYPERPARAMETER
    # Width of the margin around the previous polynomial to search
    margin = 55

    left_fit, right_fit = fit_poly(binary_warped)
    
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # If no lines found, perform search again
    if (leftx.size == 0 or rightx.size == 0):
        return fit_poly(binary_warped)
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    # cv2.imwrite('output_images/test3_identify_lane_line.jpg', result)
    # Plot the polynomial lines onto the image
    # plt.plot(left_fitx, ploty, color='yellow')
    # plt.plot(right_fitx, ploty, color='yellow')
    
    return left_fit, right_fit, result

def track_line(warped, window_width, window_height):
    margin = 25
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    smooth_factor = 12
    
    recent_centers = []      
    window_centroids = [] #Store the window centroid positions
    window = np.ones(window_width) # Window template that will be used for convolution

    # Sum quater bottom of image to get slice
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window, l_sum)) - window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window, r_sum)) - window_width/2 + int(warped.shape[1]/2)

    # Add first layer
    window_centroids.append((l_center, r_center))

    # Go through each layer looking for maximum pixel locations
    for level in range(1, (int)(warped.shape[0]/window_height)):
        # Convolve the window into vertical slice
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find best left centroid using past left center
        # Use window_width/2 as offset as signal reference is on right side
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin, warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset

        # Find best right centroid by using past right center
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin, warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset

        # Add layer
        window_centroids.append((l_center, r_center))

    recent_centers.append(window_centroids)

    # Return average values of the line centers
    return np.average(recent_centers[-smooth_factor:], axis=0)


# Line class to store fit values

class SingleLine:
    def __init__(self):
        # Line was detected previously or not?
        self.detected = False
        # Store previous fits
        self.previous_fits = []
        # Store polynomial coefficients for current fit
        self.current_fit = np.array([0, 0, 0])
        # Store polynomial coefficients for previous fit
        self.previous_fit = np.array([0, 0, 0])
        # Vehicle posistion
        self.line_base_pos = None
        # Previous vehicle posistion, it will be used if validity check fails
        self.previous_line_base_pos = None
        # Store average fit
        self.average_fit = np.array([0, 0, 0])
        # Count number of frames
        self.frame_cnt = 0

# Lane class for storing and comparing width values
class Lane:
    def __init__(self):
        self.bottom_width = 0
        self.top_width = 0
        self.average_bottom_width = 0
        self.average_top_width = 0
        self.previous_bottom_widths = []
        self.previous_top_widths = []

def calculate_fit_x(img_shape, left_fit, right_fit):
    ploty = np.linspace(0, img_shape[0] - 1, img_shape[0])

    # Calculate x values
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]

    return left_fitx, right_fitx

def check_line_validity(left_line, right_line, lane):

    # Calculate widths at top and bottom
    top_width_diff = abs(lane.top_width - lane.average_top_width)
    bottom_width_diff = abs(lane.bottom_width - lane.average_bottom_width)

    # Define validity checks
    width_check_top = top_width_diff > 0.2 * lane.average_top_width or lane.top_width > 1.25 * lane.bottom_width
    # Reject if bottom width difference is more than 5 percent of average
    width_check_bottom = bottom_width_diff > 0.05 * lane.average_bottom_width
    # Reject if top or bottom width is negative
    lane_intersect_check = lane.top_width < 0.0 or lane.bottom_width < 0.0

    # Check if parameters are ok (skip for first frame)
    if (left_line.frame_cnt > 1) and (right_line.frame_cnt > 1):
        if width_check_bottom:
            result = False
        elif width_check_top:
            result = False
        elif lane_intersect_check:
            result = False
        else:
            result = True
    else:
        # For first frames
        result = True

    return result


def average_fits(img_shape, lane):
    # Average of last five frames
    n = 5
    average_fit = [0, 0, 0]

    # If fits are less than five, append current fit
    if len(lane.previous_fits) < n:
        lane.previous_fits.append(lane.current_fit)
    # If number of fits reached to maximum i.e. n, remove last and add current fit
    if len(lane.previous_fits) == n:
        lane.previous_fits.pop(n-1)
        lane.previous_fits.insert(0, lane.current_fit)

    # Now calculate the average
    if len(lane.previous_fits) > 0:
        for i in range(0, 3):
            total = 0
            for num in range(0, len(lane.previous_fits)):
                total += lane.previous_fits[num][i]
            average_fit[i] = total / len(lane.previous_fits)

    return average_fit


def average_width(img_shape, lane):
    sum_bottom = 0
    sum_top = 0
    # Average of last five frames
    n = 5
    average_bottom_width = 0
    average_top_width = 0

    if len(lane.previous_bottom_widths) < n:
        lane.previous_bottom_widths.append(lane.bottom_width)
    # If maximum limit is reached, remove last and add current value
    if len(lane.previous_bottom_widths) == n:
        lane.previous_bottom_widths.pop(n-1)
        lane.previous_bottom_widths.insert(0, lane.bottom_width)

    # Now calculate the average
    if (len(lane.previous_bottom_widths) > 0):
        for i in range(0, len(lane.previous_bottom_widths)):
            sum_bottom += lane.previous_bottom_widths[i]
        average_bottom_width = sum_bottom / len(lane.previous_bottom_widths)


    if len(lane.previous_top_widths) < n:
        lane.previous_top_widths.append(lane.top_width)
    # If amount of fits == n, remove the last element and add the current one
    if len(lane.previous_top_widths) == n:
        lane.previous_top_widths.pop(n-1)
        lane.previous_top_widths.insert(0, lane.top_width)

    # If we have enough fits, calculate the average
    if (len(lane.previous_top_widths) > 0):
        for i in range(0, len(lane.previous_top_widths)):
            sum_top = sum_top + lane.previous_top_widths[i]
        average_top_width = sum_top / len(lane.previous_top_widths)

    return average_bottom_width, average_top_width


# Create class objects
left_line = SingleLine()
right_line = SingleLine()
lane = Lane()

#for idx, fname in enumerate(images):
def process_image(img):
    #img = cv2.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    #process image
    processimage = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(30,100))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25,100))
    c_binary = color_threshold(img, sthresh=(170, 255), vthresh=(150,255))
    mag_binary = mag_thresh(img, sobel_kernel=15, mag_thresh=(50, 100))
    dir_binary = dir_threshold(img, sobel_kernel=15, thresh=(0.7, 1.3))
    
    processimage[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | (c_binary == 1)] = 1
    
    #perspective transform
    img_size = (img.shape[1], img.shape[0])
    
    # Values for source rectangle
    bottom_left = [220,720]
    bottom_right = [1110, 720]
    top_left = [570, 470]
    top_right = [722, 470]
    src = np.float32([bottom_left,bottom_right,top_right,top_left])
    
    # Values for destination rectangle
    bottom_left = [320,720]
    bottom_right = [920, 720]
    top_left = [320, 1]
    top_right = [920, 1]
    dst = np.float32([bottom_left,bottom_right,top_right,top_left])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(processimage, M, img_size, flags=cv2.INTER_LINEAR)
    
    histogram = hist(warped)
    
    left_points, right_points, out_img = search_around_poly(warped)
    # Save points
    left_line.current_fit = left_points
    right_line.current_fit = right_points

    
    # Calculate base and mid line position
    left_line.line_base_pos = left_line.current_fit[0] * (warped.shape[0] - 1) ** 2 + left_line.current_fit[1] * (warped.shape[0] - 1) + left_line.current_fit[2]
    right_line.line_base_pos = right_line.current_fit[0] * (warped.shape[0] - 1) ** 2 + right_line.current_fit[1] * (warped.shape[0] - 1) + right_line.current_fit[2]
    left_line.line_mid_pos = left_line.current_fit[0] * (warped.shape[0] // 2) ** 2 + left_line.current_fit[1] * (warped.shape[0] // 2) + left_line.current_fit[2]
    right_line.line_mid_pos = right_line.current_fit[0] * (warped.shape[0] // 2) ** 2 + right_line.current_fit[1] * (warped.shape[0] // 2) + right_line.current_fit[2]

    # For validity check, calculate lane top, middle and bottom widths
    lane.top_width = right_line.current_fit[2] - left_line.current_fit[2]
    lane.bottom_width = right_line.line_base_pos - left_line.line_base_pos
    lane.middle_width = right_line.line_mid_pos - left_line.line_mid_pos

    # Now check if the values are valid
    if check_line_validity(left_line, right_line, lane) is False:
            left_line.current_fit = left_line.previous_fit
            right_line.current_fit = right_line.previous_fit
            left_line.line_base_pos = left_line.previous_line_base_pos
            right_line.line_base_pos = right_line.previous_line_base_pos
            left_line.detected = False
            right_line.detected = False

    else:
        # If values are valid use the values and store that lines are detected correctly
        left_line.detected = True
        right_line.detected = True
        left_line.previous_line_base_pos = left_line.line_base_pos
        right_line.previous_line_base_pos = right_line.line_base_pos
        left_line.frame_cnt += 1
        right_line.frame_cnt += 1

    
    # Calculate average for lines
    left_line.average_fit = average_fits(warped.shape, left_line)
    right_line.average_fit = average_fits(warped.shape, right_line)
    # Calculate average for lane widths
    lane.average_bottom_width, lane.average_top_width = average_width(warped.shape, lane)   
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    ploty = np.linspace(0, warped.shape[0] - 1, warped.shape[0])
    
    # Calculate x values
    left_fit_x, right_fit_x = calculate_fit_x(warped.shape, left_line.average_fit, right_line.average_fit)
    
    pts_left = np.array([np.transpose(np.vstack([left_fit_x, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fit_x, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))
    result = cv2.addWeighted(img, 1.0, newwarp, 0.3, 0.0)
    
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Measure Curvature

    # Fit new polynomials
    y_eval = np.max(ploty)
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fit_x*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, right_fit_x * xm_per_pix, 2)
    
    left_curved  = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curved  = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    mean_curve = (left_curved + right_curved)/2
    # center_diff = abs(camera_center - lane_center)
    # side_pos = 'left'
    # if center_diff <= 0:
    #    side_pos = 'right'
    
    ## Image mid horizontal position 
    mid_imgx = img.shape[1]//2
        
    ## Car position with respect to the lane
    car_pos = (left_line.line_base_pos + right_line.line_base_pos)/2
    
    ## Horizontal car offset 
    offsetx = (mid_imgx - car_pos) * xm_per_pix
    
    # Draw the text showing curvature, offset and speed
    cv2.putText(result, 'Radius of the Curvature = ' + str(round(mean_curve, 3))+'(m)',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result, 'Vehicle is '+str(abs(round(offsetx, 3)))+'m of center', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
  
    # Set current values as previous values for next frame
    left_line.previous_fit = left_line.current_fit
    right_line.previous_fit = right_line.current_fit

    # Reset / empty current fit
    left_line.current_fit = [np.array([False])]
    right_line.current_fit = [np.array([False])]
    
    return result

Output_video = 'output_video.mp4'
Input_video = 'project_video.mp4'

clip = VideoFileClip(Input_video)
video_clip = clip.fl_image(process_image)
video_clip.write_videofile(Output_video, audio = False)



[MoviePy] >>>> Building video output_video.mp4
[MoviePy] Writing video output_video.mp4





  0%|          | 0/1261 [00:00<?, ?it/s]


  0%|          | 1/1261 [00:00<06:57,  3.02it/s]


  0%|          | 2/1261 [00:00<07:13,  2.90it/s]


  0%|          | 3/1261 [00:01<07:11,  2.91it/s]


  0%|          | 4/1261 [00:01<07:08,  2.93it/s]


  0%|          | 5/1261 [00:01<07:07,  2.94it/s]


  0%|          | 6/1261 [00:02<07:04,  2.96it/s]


  1%|          | 7/1261 [00:02<07:04,  2.95it/s]


  1%|          | 8/1261 [00:02<07:04,  2.95it/s]


  1%|          | 9/1261 [00:03<07:04,  2.95it/s]


  1%|          | 10/1261 [00:03<07:04,  2.95it/s]


  1%|          | 11/1261 [00:03<07:03,  2.95it/s]


  1%|          | 12/1261 [00:04<07:01,  2.96it/s]


  1%|          | 13/1261 [00:04<07:03,  2.94it/s]


  1%|          | 14/1261 [00:04<07:00,  2.96it/s]


  1%|          | 15/1261 [00:05<06:59,  2.97it/s]


  1%|▏         | 16/1261 [00:05<07:00,  2.96it/s]


  1%|▏         | 17/1261 [00:05<06:59,  2.96it/s]


  1%|▏         | 18/1261 [00:06<06:58,  2.97it/s]


  2%|▏         | 19/1261 [0

 11%|█         | 134/1261 [00:52<07:47,  2.41it/s]


 11%|█         | 135/1261 [00:53<07:53,  2.38it/s]


 11%|█         | 136/1261 [00:53<07:43,  2.43it/s]


 11%|█         | 137/1261 [00:54<07:48,  2.40it/s]


 11%|█         | 138/1261 [00:54<07:36,  2.46it/s]


 11%|█         | 139/1261 [00:54<07:43,  2.42it/s]


 11%|█         | 140/1261 [00:55<07:34,  2.47it/s]


 11%|█         | 141/1261 [00:55<07:37,  2.45it/s]


 11%|█▏        | 142/1261 [00:56<07:40,  2.43it/s]


 11%|█▏        | 143/1261 [00:56<07:42,  2.42it/s]


 11%|█▏        | 144/1261 [00:56<07:41,  2.42it/s]


 11%|█▏        | 145/1261 [00:57<07:44,  2.40it/s]


 12%|█▏        | 146/1261 [00:57<07:44,  2.40it/s]


 12%|█▏        | 147/1261 [00:58<07:43,  2.40it/s]


 12%|█▏        | 148/1261 [00:58<07:43,  2.40it/s]


 12%|█▏        | 149/1261 [00:59<07:44,  2.40it/s]


 12%|█▏        | 150/1261 [00:59<07:47,  2.38it/s]


 12%|█▏        | 151/1261 [00:59<07:35,  2.43it/s]


 12%|█▏        | 152/1261 [01:00<07:37,  2.43i

 32%|███▏      | 398/1261 [02:43<05:50,  2.46it/s]


 32%|███▏      | 399/1261 [02:43<05:54,  2.43it/s]


 32%|███▏      | 400/1261 [02:43<05:48,  2.47it/s]


 32%|███▏      | 401/1261 [02:44<05:50,  2.45it/s]


 32%|███▏      | 402/1261 [02:44<05:46,  2.48it/s]


 32%|███▏      | 403/1261 [02:45<05:54,  2.42it/s]


 32%|███▏      | 404/1261 [02:45<05:47,  2.47it/s]


 32%|███▏      | 405/1261 [02:45<05:53,  2.42it/s]


 32%|███▏      | 406/1261 [02:46<05:48,  2.46it/s]


 32%|███▏      | 407/1261 [02:46<05:53,  2.41it/s]


 32%|███▏      | 408/1261 [02:47<05:47,  2.45it/s]


 32%|███▏      | 409/1261 [02:47<05:52,  2.42it/s]


 33%|███▎      | 410/1261 [02:47<05:45,  2.46it/s]


 33%|███▎      | 411/1261 [02:48<05:49,  2.43it/s]


 33%|███▎      | 412/1261 [02:48<05:44,  2.47it/s]


 33%|███▎      | 413/1261 [02:49<05:50,  2.42it/s]


 33%|███▎      | 414/1261 [02:49<05:44,  2.46it/s]


 33%|███▎      | 415/1261 [02:50<05:49,  2.42it/s]


 33%|███▎      | 416/1261 [02:50<05:43,  2.46i

 52%|█████▏    | 662/1261 [04:33<04:13,  2.36it/s]


 53%|█████▎    | 663/1261 [04:33<04:17,  2.33it/s]


 53%|█████▎    | 664/1261 [04:34<04:13,  2.36it/s]


 53%|█████▎    | 665/1261 [04:34<04:17,  2.32it/s]


 53%|█████▎    | 666/1261 [04:34<04:11,  2.36it/s]


 53%|█████▎    | 667/1261 [04:35<04:12,  2.35it/s]


 53%|█████▎    | 668/1261 [04:35<04:23,  2.25it/s]


 53%|█████▎    | 669/1261 [04:36<04:19,  2.28it/s]


 53%|█████▎    | 670/1261 [04:36<04:18,  2.29it/s]


 53%|█████▎    | 671/1261 [04:37<04:18,  2.28it/s]


 53%|█████▎    | 672/1261 [04:37<04:13,  2.32it/s]


 53%|█████▎    | 673/1261 [04:38<04:16,  2.30it/s]


 53%|█████▎    | 674/1261 [04:38<04:15,  2.30it/s]


 54%|█████▎    | 675/1261 [04:38<04:07,  2.37it/s]


 54%|█████▎    | 676/1261 [04:39<04:09,  2.34it/s]


 54%|█████▎    | 677/1261 [04:39<04:09,  2.34it/s]


 54%|█████▍    | 678/1261 [04:40<04:13,  2.30it/s]


 54%|█████▍    | 679/1261 [04:40<04:06,  2.36it/s]


 54%|█████▍    | 680/1261 [04:41<04:11,  2.31i

 73%|███████▎  | 926/1261 [06:21<02:17,  2.44it/s]


 74%|███████▎  | 927/1261 [06:22<02:18,  2.40it/s]


 74%|███████▎  | 928/1261 [06:22<02:16,  2.45it/s]


 74%|███████▎  | 929/1261 [06:22<02:20,  2.36it/s]


 74%|███████▍  | 930/1261 [06:23<02:20,  2.36it/s]


 74%|███████▍  | 931/1261 [06:23<02:19,  2.36it/s]


 74%|███████▍  | 932/1261 [06:24<02:16,  2.41it/s]


 74%|███████▍  | 933/1261 [06:24<02:17,  2.38it/s]


 74%|███████▍  | 934/1261 [06:25<02:17,  2.38it/s]


 74%|███████▍  | 935/1261 [06:25<02:17,  2.37it/s]


 74%|███████▍  | 936/1261 [06:25<02:14,  2.42it/s]


 74%|███████▍  | 937/1261 [06:26<02:17,  2.35it/s]


 74%|███████▍  | 938/1261 [06:26<02:14,  2.40it/s]


 74%|███████▍  | 939/1261 [06:27<02:15,  2.38it/s]


 75%|███████▍  | 940/1261 [06:27<02:12,  2.43it/s]


 75%|███████▍  | 941/1261 [06:27<02:12,  2.41it/s]


 75%|███████▍  | 942/1261 [06:28<02:10,  2.44it/s]


 75%|███████▍  | 943/1261 [06:28<02:13,  2.38it/s]


 75%|███████▍  | 944/1261 [06:29<02:10,  2.42i

 94%|█████████▍| 1187/1261 [08:13<00:30,  2.40it/s]


 94%|█████████▍| 1188/1261 [08:14<00:30,  2.36it/s]


 94%|█████████▍| 1189/1261 [08:14<00:29,  2.41it/s]


 94%|█████████▍| 1190/1261 [08:14<00:30,  2.37it/s]


 94%|█████████▍| 1191/1261 [08:15<00:29,  2.41it/s]


 95%|█████████▍| 1192/1261 [08:15<00:29,  2.36it/s]


 95%|█████████▍| 1193/1261 [08:16<00:28,  2.41it/s]


 95%|█████████▍| 1194/1261 [08:16<00:28,  2.36it/s]


 95%|█████████▍| 1195/1261 [08:16<00:27,  2.41it/s]


 95%|█████████▍| 1196/1261 [08:17<00:27,  2.37it/s]


 95%|█████████▍| 1197/1261 [08:17<00:26,  2.43it/s]


 95%|█████████▌| 1198/1261 [08:18<00:26,  2.38it/s]


 95%|█████████▌| 1199/1261 [08:18<00:25,  2.43it/s]


 95%|█████████▌| 1200/1261 [08:19<00:25,  2.38it/s]


 95%|█████████▌| 1201/1261 [08:19<00:24,  2.43it/s]


 95%|█████████▌| 1202/1261 [08:19<00:25,  2.35it/s]


 95%|█████████▌| 1203/1261 [08:20<00:24,  2.36it/s]


 95%|█████████▌| 1204/1261 [08:20<00:24,  2.33it/s]


 96%|█████████▌| 1205/1261 [

[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video.mp4 



In [6]:
!git add .\

On branch master
Your branch is behind 'origin/master' by 2 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   examples/.ipynb_checkpoints/example-checkpoint.ipynb
	modified:   examples/example.ipynb
	modified:   set_git.sh
	deleted:    writeup_template.md

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	Final.html
	Final.ipynb
	__pycache__/
	camera_cal/calibration1_undistorted.jpg
	camera_cal/calibration_pickle.p
	examples/corners_found0.jpg
	examples/corners_found10.jpg
	examples/corners_found12.jpg
	examples/corners_found13.jpg
	examples/corners_found14.jpg
	examples/corners_found15.jpg
	examples/corners_found16.jpg
	examples/corners_found17.jpg
	examples/corners_found18.jpg
	examples/corners_found19.jpg
	examples